In [1]:
import pandas as pd
import psycopg2
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
import sys
# import random
# import implicit
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import pickle
# from lightfm.data import Dataset
import csv
from cmfrec import CMF

'''这个代码比较不同聚类方式对冷启动的影响'''

'''这一块代码是数据处理'''
original_data = pd.read_csv('/home/wyf/Desktop/tmall/data.csv') # 在cold start里用
# original_data = original_data.dropna()
original_data.drop(original_data.columns[0],axis=1,inplace=True)
original_data['user_id'] = original_data.user_id.astype(int)

retail_data = pd.read_csv('/home/wyf/Desktop/tmall/train_cold.csv')
retail_data.drop(retail_data.columns[0],axis=1,inplace=True)

retail_data['user_id'] = retail_data.user_id.astype(int)
data = retail_data[['user_id', 'item_id', 'Occurrence']] # Get rid of unnecessary info

item_lookup = retail_data[['item_id', 'cat_id']].drop_duplicates() # Only get unique item/description pairs
item_lookup['item_id'] = retail_data.item_id.astype(int) # Encode as strings for future lookup ease

grouped_cleaned = data.groupby(['user_id', 'item_id']).sum().reset_index() # Group together
grouped_cleaned.Occurrence.loc[grouped_cleaned.Occurrence == 0] = 1
grouped_purchased = grouped_cleaned.query('Occurrence > 0')

customers = list(np.sort(grouped_purchased.user_id.unique())) # Get our unique customers
products = list(grouped_purchased.item_id.unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased.Occurrence)

original_data.head()

/home/wyf/.local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,Occurrence
0,234512,179830,1208,546,2276,1111,2,5.0,0.0,1
1,217494,646334,1208,1304,2276,1111,2,4.0,1.0,2
2,278823,18090,1208,1304,2276,1111,2,6.0,1.0,4
3,271127,539608,1208,422,2276,1111,2,4.0,2.0,1
4,131687,717816,629,3912,2276,1010,2,4.0,0.0,1


In [2]:
grouped_purchased.head()

,user_id,item_id,Occurrence
0,2,293244,1
1,2,917794,1
2,5,815566,1
3,14,267067,2
4,14,349999,2


In [3]:
with open ('/home/wyf/Desktop/tmall/test_cold.txt', 'rb') as fp:
    test_set = pickle.load(fp) 
test_set[:5]

[(412232, 888618),
 (359519, 295260),
 (8765, 717906),
 (38288, 815566),
 (16852, 252098)]

In [4]:
retail_data.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,Occurrence
0,234512,179830,1208,546,2276,1111,2,5.0,0.0,1
1,278823,18090,1208,1304,2276,1111,2,6.0,1.0,4
2,271127,539608,1208,422,2276,1111,2,4.0,2.0,1
3,131687,717816,629,3912,2276,1010,2,4.0,0.0,1
4,333647,1014969,1213,4144,2276,812,2,4.0,0.0,1


In [5]:
def pop_goods(data, num):
    
    '''找出买的最多的前num个商品'''
    data = data.copy()
    data['total_occ'] = data.groupby(['item_id']).cumcount(ascending=False)+1 # here every item appears only once to everybody
    data = data.drop_duplicates(subset=['item_id'], keep='first')
    goods = list(data.sort_values('total_occ', ascending=False).head(num).item_id)
    
    return goods
# popular = pop_goods(retail_data.copy(), 100)
# popular[:5]

In [6]:
grouped_purchased.columns = ['UserId','ItemId','Rating']
grouped_purchased.head()

,UserId,ItemId,Rating
0,2,293244,1
1,2,917794,1
2,5,815566,1
3,14,267067,2
4,14,349999,2


In [7]:
def accuracy(item, _list):
    if item in _list:
        return 1
    else:
        return 0

In [8]:
def acc_topN_with_cat_pop(changed_pair, num, data):
    popular = pd.DataFrame(pop_goods(data, 100))
#     print(len(popular))
    count = 0
#     print(altered_pair)
    for i in range(len(changed_pair)):
        popular.columns = ['item_id']
        _item_cat = pd.merge(popular, item_lookup)
#         print(_item_cat.shape)
#         print(_item_cat)
#         print(_item_cat)
#         item_cat = _item_cat.drop_duplicates(subset=['item_id'])
#         print(item_cat)
        cat_topN = list(_item_cat.groupby('cat_id').head(num).item_id)
#         print(cat_topN[:5])
#         print(len(cat_topN))
        count = count + accuracy(changed_pair[i][1], cat_topN)

    return count/len(changed_pair)

topN_acc = acc_topN_with_cat_pop(test_set, 5, data.copy())
topN_acc

0.4138423846467946

In [9]:
user = test_set[0][0]
user

412232

In [10]:
user_gender = original_data.gender[original_data.user_id==user].item()
user_gender
# original_data[original_data.gender==0]

1.0

In [11]:
gender0 = retail_data[retail_data.gender==0]
gender1 = retail_data[retail_data.gender==1]
gender2 = retail_data[retail_data.gender==2]
popular0 = pd.DataFrame(pop_goods(gender0, 100))
popular1 = pd.DataFrame(pop_goods(gender1, 100))
popular2 = pd.DataFrame(pop_goods(gender2, 100))
print(popular0[:10])
print(popular1[:10])
print(popular2[:10])

         0
0  1039919
1   179830
2   195714
3   951042
4  1029992
5   293244
6    18090
7   939915
8  1058188
9   969584
         0
0   195714
1  1029992
2  1039919
3   179830
4   590204
5   718528
6   588327
7    74158
8    18090
9  1058188
         0
0   195714
1   293244
2  1029992
3  1039919
4   179830
5   718528
6   590204
7    74158
8   969584
9   588327


In [12]:
gender0.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,Occurrence
0,234512,179830,1208,546,2276,1111,2,5.0,0.0,1
3,131687,717816,629,3912,2276,1010,2,4.0,0.0,1
4,333647,1014969,1213,4144,2276,812,2,4.0,0.0,1
5,333647,417530,1213,4144,2276,812,2,4.0,0.0,1
6,333647,1041773,267,4144,2276,812,2,4.0,0.0,1


In [13]:
gender1.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,Occurrence
1,278823,18090,1208,1304,2276,1111,2,6.0,1.0,4
7,241784,325774,1028,3912,2276,1111,2,5.0,1.0,1
8,217844,526817,1208,4144,2276,1002,2,3.0,1.0,1
9,217844,179830,1208,546,2276,1020,2,3.0,1.0,1
15,133003,38250,1208,1719,2276,1105,2,4.0,1.0,1


In [14]:
gender2.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,Occurrence
2,271127,539608,1208,422,2276,1111,2,4.0,2.0,1
11,118255,956327,602,1200,2276,605,2,7.0,2.0,1
42,3025,74158,267,1200,2276,812,2,4.0,2.0,1
43,3025,588327,1213,1200,2276,812,2,4.0,2.0,1
44,3025,1029992,267,1200,2276,812,2,4.0,2.0,1


In [15]:
def age_segment(data, age):
    age_seg = data[data.age_range==age]
    return age_seg

def gender_segment(data, gender):
    gender_seg = data[data.gender==gender]
    return gender_seg

In [16]:
len(test_set)

4898

In [17]:
test_set[:5]

[(412232, 888618),
 (359519, 295260),
 (8765, 717906),
 (38288, 815566),
 (16852, 252098)]

In [19]:
def acc_topN_with_cat_pop_segement(changed_pair, num, ori_data, train_data):
#     gender0 = train_data[train_data.gender==0.0]
#     gender1 = train_data[train_data.gender==1.0]
#     gender2 = train_data[train_data.gender==2.0]
#     popular0 = pd.DataFrame(pop_goods(gender0, 100))
#     popular1 = pd.DataFrame(pop_goods(gender1, 100))
#     popular2 = pd.DataFrame(pop_goods(gender2, 100))
#     print(popular0[:5])
#     print(popular1[:5])
#     print(popular2[:5])
#     print(len(popular))
    count = 0
#     print(altered_pair)
    for i in range(len(changed_pair)):
#         print(i)
#         print(ori_data[ori_data.user_id==changed_pair[i][0]])
        # 得到用户的年龄和性别
        user_gender = ori_data.gender[ori_data.user_id==changed_pair[i][0]].item()
#         user_age = ori_data.age_range[ori_data.user_id==changed_pair[i][0]].item()
        
        # 选择出和用户性别一样的用户集 gender_seg
        gender_seg = gender_segment(ori_data, user_gender)
        # 在gender_seg中选出和用户年龄一样的用户集
#         age_seg = age_segment(gender_seg, user_age)

        popular = pd.DataFrame(pop_goods(gender_seg, 100))
        popular.columns = ['item_id']
        _item_cat = pd.merge(popular, item_lookup)
#         if user_gender == 0:            
# #             popular0.columns = ['item_id']
#             _item_cat = pd.merge(popular, item_lookup)
#         elif user_gender == 1:
# #             popular1.columns = ['item_id']
#             _item_cat = pd.merge(popular, item_lookup)
#         elif user_gender == 2:
# #             popular2.columns = ['item_id']
            
#         print(_item_cat)
#         print(_item_cat)
#         item_cat = _item_cat.drop_duplicates(subset=['item_id'])
#         print(item_cat)
        cat_topN = list(_item_cat.groupby('cat_id').head(num).item_id)
#         print(cat_topN[:5])
#         print(len(cat_topN))
        count = count + accuracy(changed_pair[i][1], cat_topN)

    return count/len(changed_pair)

topN_acc = acc_topN_with_cat_pop_segement(test_set, 5, original_data.copy(), retail_data.copy())
topN_acc

0.42282564311964066

In [20]:
def acc_topN_with_cat_pop_segement(changed_pair, num, ori_data, train_data):
#     gender0 = train_data[train_data.gender==0.0]
#     gender1 = train_data[train_data.gender==1.0]
#     gender2 = train_data[train_data.gender==2.0]
#     popular0 = pd.DataFrame(pop_goods(gender0, 100))
#     popular1 = pd.DataFrame(pop_goods(gender1, 100))
#     popular2 = pd.DataFrame(pop_goods(gender2, 100))
#     print(popular0[:5])
#     print(popular1[:5])
#     print(popular2[:5])
#     print(len(popular))
    count = 0
#     print(altered_pair)
    for i in range(len(changed_pair)):
#         print(i)
#         print(ori_data[ori_data.user_id==changed_pair[i][0]])
        # 得到用户的年龄和性别
#         user_gender = ori_data.gender[ori_data.user_id==changed_pair[i][0]].item()
        user_age = ori_data.age_range[ori_data.user_id==changed_pair[i][0]].item()
        
        # 选择出和用户性别一样的用户集 gender_seg
#         gender_seg = gender_segment(ori_data, user_gender)
        # 在gender_seg中选出和用户年龄一样的用户集
        age_seg = age_segment(ori_data, user_age)

        popular = pd.DataFrame(pop_goods(age_seg, 100))
        popular.columns = ['item_id']
        _item_cat = pd.merge(popular, item_lookup)
#         if user_gender == 0:            
# #             popular0.columns = ['item_id']
#             _item_cat = pd.merge(popular, item_lookup)
#         elif user_gender == 1:
# #             popular1.columns = ['item_id']
#             _item_cat = pd.merge(popular, item_lookup)
#         elif user_gender == 2:
# #             popular2.columns = ['item_id']
            
#         print(_item_cat)
#         print(_item_cat)
#         item_cat = _item_cat.drop_duplicates(subset=['item_id'])
#         print(item_cat)
        cat_topN = list(_item_cat.groupby('cat_id').head(num).item_id)
#         print(cat_topN[:5])
#         print(len(cat_topN))
        count = count + accuracy(changed_pair[i][1], cat_topN)

    return count/len(changed_pair)

topN_acc = acc_topN_with_cat_pop_segement(test_set, 5, original_data.copy(), retail_data.copy())
topN_acc

0.42057982850142916

In [18]:
def acc_topN_with_cat_pop_segement(changed_pair, num, ori_data, train_data):
#     gender0 = train_data[train_data.gender==0.0]
#     gender1 = train_data[train_data.gender==1.0]
#     gender2 = train_data[train_data.gender==2.0]
#     popular0 = pd.DataFrame(pop_goods(gender0, 100))
#     popular1 = pd.DataFrame(pop_goods(gender1, 100))
#     popular2 = pd.DataFrame(pop_goods(gender2, 100))
#     print(popular0[:5])
#     print(popular1[:5])
#     print(popular2[:5])
#     print(len(popular))
    count = 0
#     print(altered_pair)
    for i in range(len(changed_pair)):
#         print(i)
#         print(ori_data[ori_data.user_id==changed_pair[i][0]])
        # 得到用户的年龄和性别
        user_gender = ori_data.gender[ori_data.user_id==changed_pair[i][0]].item()
        user_age = ori_data.age_range[ori_data.user_id==changed_pair[i][0]].item()
        
        # 选择出和用户性别一样的用户集 gender_seg
        gender_seg = gender_segment(ori_data, user_gender)
        # 在gender_seg中选出和用户年龄一样的用户集
        age_seg = age_segment(gender_seg, user_age)

        popular = pd.DataFrame(pop_goods(age_seg, 100))
        popular.columns = ['item_id']
        _item_cat = pd.merge(popular, item_lookup)
#         if user_gender == 0:            
# #             popular0.columns = ['item_id']
#             _item_cat = pd.merge(popular, item_lookup)
#         elif user_gender == 1:
# #             popular1.columns = ['item_id']
#             _item_cat = pd.merge(popular, item_lookup)
#         elif user_gender == 2:
# #             popular2.columns = ['item_id']
            
#         print(_item_cat)
#         print(_item_cat)
#         item_cat = _item_cat.drop_duplicates(subset=['item_id'])
#         print(item_cat)
        cat_topN = list(_item_cat.groupby('cat_id').head(num).item_id)
#         print(cat_topN[:5])
#         print(len(cat_topN))
        count = count + accuracy(changed_pair[i][1], cat_topN)

    return count/len(changed_pair)

topN_acc = acc_topN_with_cat_pop_segement(test_set, 5, original_data.copy(), retail_data.copy())
topN_acc

0.4322172315230706